# S1. Red convolucional para MNIST



Como en la anterior práctica, primero importamos el conjunto de MNIST y lo normalizamos, pero sin convertir las imágenes en vectores unidimensionales, ya que vamos a trabajar con redes convolucionales que explotan la estructura 2D de las imágenes.

In [9]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 28, 28)
val set (12000, 28, 28)


## Modelo base
 Partiremos de una topología base que toma la red MLP de la última sesión de la primera práctica (dos capas densas de 1024 neuronas), y le incorpora un par de capas convolucionales cada una seguida por average pooling inspirada en la arquitectura LeNet (1998) propuesta por Yann LeCun para MNIST. 
 

In [2]:
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
373/375 [============================>.] - ETA: 0s - loss: 0.2652 - accuracy: 0.9224
Epoch 1: val_accuracy improved from -inf to 0.97275, saving model to best_model.h5
375/375 [==============================] - 5s 11ms/step - loss: 0.2645 - accuracy: 0.9226 - val_loss: 0.0914 - val_accuracy: 0.9728 - lr: 0.0010
Epoch 2/25
 10/375 [..............................] - ETA: 4s - loss: 0.0770 - accuracy: 0.9789

/home/pegi/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - ETA: 0s - loss: 0.0749 - accuracy: 0.9770
Epoch 2: val_accuracy improved from 0.97275 to 0.98433, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.0749 - accuracy: 0.9770 - val_loss: 0.0557 - val_accuracy: 0.9843 - lr: 0.0010
Epoch 3/25
374/375 [============================>.] - ETA: 0s - loss: 0.0513 - accuracy: 0.9838
Epoch 3: val_accuracy did not improve from 0.98433
375/375 [==============================] - 4s 11ms/step - loss: 0.0513 - accuracy: 0.9838 - val_loss: 0.0530 - val_accuracy: 0.9843 - lr: 0.0010
Epoch 4/25
373/375 [============================>.] - ETA: 0s - loss: 0.0396 - accuracy: 0.9876
Epoch 4: val_accuracy improved from 0.98433 to 0.98592, saving model to best_model.h5
375/375 [==============================] - 5s 15ms/step - loss: 0.0396 - accuracy: 0.9876 - val_loss: 0.0502 - val_accuracy: 0.9859 - lr: 0.0010
Epoch 5/25
375/375 [==============================] - ETA: 0s - l

## Ejercicio:

Probar las técnicas presentadas en la práctica 1 (dropout, batchnorm y aumento de datos) para obtener un acierto en test > 99%, incluso mejor que la obtenida con redes MLP.

In [10]:
from keras import Sequential
from keras.layers import Input, Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = Sequential()

model.add(Input((28,28,1)))
model.add(Conv2D(filters=6, kernel_size=(5,5), activation='relu', input_shape=(28,28,1)))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2), strides=2))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=10, activation = 'softmax'))

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=25
batch_size=128

history = model.fit(x_train, y_train, batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Epoch 1/25
373/375 [============================>.] - ETA: 0s - loss: 0.1671 - accuracy: 0.9483
Epoch 1: val_accuracy improved from -inf to 0.93042, saving model to best_model.h5
375/375 [==============================] - 7s 14ms/step - loss: 0.1669 - accuracy: 0.9484 - val_loss: 0.3761 - val_accuracy: 0.9304 - lr: 0.0010
Epoch 2/25
374/375 [============================>.] - ETA: 0s - loss: 0.0655 - accuracy: 0.9798
Epoch 2: val_accuracy improved from 0.93042 to 0.98467, saving model to best_model.h5
375/375 [==============================] - 5s 14ms/step - loss: 0.0655 - accuracy: 0.9798 - val_loss: 0.0554 - val_accuracy: 0.9847 - lr: 0.0010
Epoch 3/25
375/375 [==============================] - ETA: 0s - loss: 0.0481 - accuracy: 0.9852
Epoch 3: val_accuracy did not improve from 0.98467
375/375 [==============================] - 5s 14ms/step - loss: 0.0481 - accuracy: 0.9852 - val_loss: 0.0580 - val_accuracy: 0.9847 - lr: 0.0010
Epoch 4/25
373/375 [============================>.] - ETA